Code portions:
- All Pale chapters (through 23.1)
- Reasonably "optimal" preprocessing parameters
- fine-tuned embedding model
- ElasticSearch container Document Store
- Iterate and add other series embeddings?

In [1]:
%%bash

nvidia-smi


Tue Mar  7 21:10:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A    0C    P0    N/A /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import pickle
import logging
import time
from haystack.document_stores import PineconeDocumentStore
from haystack import Document
from haystack.nodes import PreProcessor, EmbeddingRetriever, Seq2SeqGenerator, TransformersSummarizer, FARMReader
from haystack.pipelines import GenerativeQAPipeline, ExtractiveQAPipeline, SearchSummarizationPipeline


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.WARNING)


In [4]:
# print(os.getcwd())
# os.chdir('./drive/MyDrive/pale-companion-files/es-experimentation/')

# print(os.getcwd())

In [5]:
MODEL_LOCATION = "../../twig_otherverse_parahumans_adapted/"
PALE_CHAPTERS_FNAME = "../../data/chapter_fmt_list.pkl"

In [6]:
with open(PALE_CHAPTERS_FNAME,'rb') as f:
    all_chapters = pickle.load(f)
chapters = [i for i in all_chapters if int(i['meta']['arc_number']) < 3]
print(f"Testing with {len(chapters)} chapters")

Testing with 30 chapters


In [7]:
chapters[0]['meta']

{'arc_title': 'Blood Runs Cold',
 'pov': 'Louise',
 'wordcount': '7174',
 'series_chapter_number': 1,
 'arc_number': '0',
 'extra_material': False,
 'title': 'Blood Runs Cold - 0.0',
 'chapter': '0.0'}

In [8]:
preprocessor = PreProcessor(
    split_by='word',
    split_length=250,
    split_overlap=20,

    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_respect_sentence_boundary=True,
    progress_bar=True, 
    add_page_number=True
)
docs = preprocessor.process(chapters)
print(f"We will be working with {len(docs)} documents from {len(all_chapters)} chapters")

Preprocessing:   0%|          | 0/30 [00:00<?, ?docs/s]

We will be working with 1038 documents from 307 chapters


In [9]:
document_store = PineconeDocumentStore(
    api_key='6dcb03dd-509b-4ad3-8342-b705027cb07c',
    similarity='cosine',
    duplicate_documents='skip',
    embedding_field="emb",
    embedding_dim=768,
    metadata_config = {'indexed':['series_chapter_number','chapters','title']}
)

document_store.write_documents(docs, batch_size=2)

Writing Documents:   0%|          | 0/1038 [00:00<?, ?it/s]

In [10]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=MODEL_LOCATION,
    model_format="sentence_transformers",
    max_seq_len=500,
    progress_bar=True,
)


document_store.update_embeddings(retriever,update_existing_embeddings=False, batch_size=2)

Updating Embedding:   0%|          | 0/1038 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa", num_beams=8, max_length=500)
# Eventually with want to tweak the above
pipe = GenerativeQAPipeline(generator, retriever) # We specify the params later

In [ ]:
TEST_QUESTION = "Who is Avery Kelly?"

TEST_OUTPUT = pipe.run(
                query = TEST_QUESTION,
                params = {
                    "Retriever": {"top_k":10},
                    # "Generator": {"max_length": maxlength, "num_beams": nbeam}
                }
            )

In [ ]:
TEST_OUTPUT